<div align="center">

  <a href="https://clear.ml" target="_blank">
    <img width="512", src="https://github.com/allegroai/clearml/raw/master/docs/clearml-logo.svg"></a>


<br>

<h1>Notebook 3: Remote Task Execution</h1>

<br>

Hi there! This is the third notebook in the ClearML getting started notebook series, meant to teach you the ropes. In the <a href="https://colab.research.google.com/drive/183s0Jn2WG14zhOwCZZAZgTsbH5LeKhpA#scrollTo=LjRqze6s96Ri" target="_blank">last notebook</a> we set up an agent within the Google Colab environment. Now it's time to make that agent run our task! We're going to clone our first, exisiting, experiment, change some parameters and then run the resulting task remotely using our colab agent.

You can find out more details about the other ClearML modules and the technical specifics of each in <a href="https://clear.ml/docs" target="_blank">our documentation.</a>

<table>
<tbody>
  <tr>
    <td>Step 1: Experiment Management</td>
    <td><a target="_blank" href="https://colab.research.google.com/github/allegroai/clearml/blob/master/docs/tutorials/Getting_Started_1_Experiment_Management.ipynb">
  <img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/>
</a></td>
  </tr>
  <tr>
    <td>Step 2: Remote Agent</td>
    <td><a target="_blank" href="https://colab.research.google.com/github/allegroai/clearml/blob/master/docs/tutorials/Getting_Started_2_Setting_Up_Agent.ipynb">
  <img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/>
</a></td>
  </tr>
  <tr>
    <td><b>Step 3: Remote Task Execution</b></td>
    <td><a target="_blank" href="https://colab.research.google.com/github/allegroai/clearml/blob/master/docs/tutorials/Getting_Started_3_Remote_Execution.ipynb">
  <img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/>
</a></td>
  </tr>
</tbody>
</table>

</div>

# 📦 Setup

Just like in the other notebooks, we will need to install and setup ClearML.

**NOTE: Make sure the agent is still running in the other notebook, when running this one!**

In [ ]:
%pip install --upgrade clearml
import clearml
clearml.browser_login()

# 🔎 Querying our Task

If we want to remotely run a task, we first need to know which one!
Getting the Task ID from the webUI is quite easy, just navigate to your project, select the task you're interested in and click this button to copy the Task ID to your clipboard:

![](https://i.imgur.com/W7ZnEnX.png)


In [ ]:
# from clearml import Task

# task = Task.get_task(task_id="YOUR_TASK_ID")
# task.name, task.id

('TB Logging', '0bfefb8b86ba44798d9abe34ba0a6ab4')

However, we can also query ClearML using the Python SDK. Let's search your ClearML history for any task in the project `Getting Started` with the name `XGBoost Training` (both of which we used in tutorial notebook 1). ClearML should then give us a list of Task IDs that could fit the bill.

In [ ]:
from clearml import Task

# This will get the single most recent task fitting the description
# ⚠️ NOTE: Make sure you ran the XGBoost Training cell from notebook 1 and that the project and task name exist!
task = Task.get_task(project_name="Getting Started", task_name="XGBoost Training")

if not task:
  print("⚠️ WARNING: In order to make this work, you will need the XGBoost Training task from Notebook 1. Make sure to run the cell linked below in the same ClearML account!")
  print("https://colab.research.google.com/drive/1oHiW1qwLVvazk3qFZWBULfpciPEQp8kc#scrollTo=CSaL3XTqhYAy&line=5&uniqifier=1")
else:
  print(task.name, task.id)

XGBoost Training 1c7f708a976b413da147b341c54f862d


In [ ]:
# This will get all tasks fitting the description
tasks = Task.get_tasks(project_name="Getting Started", task_name="XGBoost Training")
[(task.name, task.id) for task in tasks]

[('XGBoost Training', '959c353c183245d5b85bfb2958e49f5d'),
 ('XGBoost Training', '6810dd7c2c944099a4bc544a4c5079ff'),
 ('XGBoost Training', '818894e032a44e7c9bafd758ee9fae7b'),
 ('XGBoost Training', '762d13fc58ad4c5e8ae3695195b00d97'),
 ('XGBoost Training', '3bf2b49800c24fd7b16f64f8c70cef83'),
 ('XGBoost Training', '95ead34d9ce94955b2998b1de37f4892'),
 ('XGBoost Training', '1c7f708a976b413da147b341c54f862d')]

In [ ]:
# Let's set our task ID based on the results here so we can use it below
task_id = task.id
task_id

'1c7f708a976b413da147b341c54f862d'

# 👥 Clone and Enqueue Task

Now that we know which task we want to run, we can start by cloning it. Cloning a task makes it editable. Now we can override any of the parameters or runtime configurations. For example, here we can set a different `max_depth` by specifying the parameter name as well as the section.

![](https://i.imgur.com/IOL20Sg.png)

In [ ]:
# Clone the original task and keep track of the new one
new_task = Task.clone(source_task=task)
# Override any of the parameters!
new_task.update_parameters({"General/max_depth": 3})
# We can even rename it if we wanted
new_task.rename(f"Cloned Task")
# Now enuque it for the colab worker to start working on it!
Task.enqueue(task=new_task, queue_name="default")

<tasks.EnqueueResponse: {
    "updated": 1,
    "fields": {
        "status": "queued",
        "status_reason": "",
        "status_message": "",
        "status_changed": "2023-02-27T15:53:37.611445+00:00",
        "last_update": "2023-02-27T15:53:37.611445+00:00",
        "last_change": "2023-02-27T15:53:37.611445+00:00",
        "last_changed_by": "fed1d02f8f5c40c79f84733ccc8ec065",
        "enqueue_status": "created",
        "execution.queue": "2d9d510d7f8546ca939555dffe45deb1"
    },
    "queued": 1
}>

In [ ]:
# Let's take a look at the status
new_task.status

'in_progress'

In [ ]:
from time import sleep
# Now we can set up a loop that waits until our task is done!
# If you have enabled notifications on Colab, it will even let you know
# when the ClearML task is done!
while new_task.status not in ["success", "failed"]:
  if new_task.status == "draft":
    print("Task is still in draft mode! You have to enqueue it before the agent can run it.")

  print(f"Task is not done yet! Current status: {new_task.status}")
  sleep(5)

When a task is enqueued, it has 4 possible states:

- Pending (the task is waiting to be pulled by a worker)
- In Progress (the task is pulled and being executed)
- Completed
- Failed

You can keep track of this status by using the `.status` property of a clearml Task instance. If you only have the Task ID, you can always get the task object by running `task = Task.get_task(task_id="your_task_id")`

In the webUI the status of a task is pretty visible:
![](https://i.imgur.com/Ep7v6AN.png)

**NOTE: The first time running a task can take quite a long time because the agent has to install all of the original packages and get the original code. Everything is cached however, so it should be faster next time!**

If everything went well, you should now see the colab worker start working on the task:

![](https://i.imgur.com/eI0ui5J.png)

# 🧐 Inspecting the results

Now that the task has completed successfully, you can now view its results just like any other experiment in the experiment manager!

![](https://i.imgur.com/zfkuKlO.png)

You can of course also select both the original and the cloned experiment and compare them directly in ClearML!

![](https://i.imgur.com/Gd56Swd.png)

Remember that the SDK is quite powerful too, so almost any piece of information that you can see in the webUI, you can also get from the SDK to build automations onto.

For example, we could get the reported scalars right into our python environment with just a single function.

In [ ]:
new_task.get_reported_scalars()

{':monitor:machine': {'cpu_usage': {'name': 'cpu_usage',
   'x': [1789, 3900, 6184, 7946],
   'y': [80.6510009765625,
    73.74199676513672,
    73.8479995727539,
    81.6760025024414]},
  'disk_free_percent': {'name': 'disk_free_percent',
   'x': [1789, 3900, 6184, 7946],
   'y': [64.30000305175781,
    64.30000305175781,
    64.30000305175781,
    64.30000305175781]},
  'io_read_mbs': {'name': 'io_read_mbs',
   'x': [1789, 3900, 6184, 7946],
   'y': [0.029999999329447746,
    0.012000000104308128,
    0.012000000104308128,
    0.01899999938905239]},
  'io_write_mbs': {'name': 'io_write_mbs',
   'x': [1789, 3900, 6184, 7946],
   'y': [4.526000022888184,
    0.8140000104904175,
    0.02800000086426735,
    0.032999999821186066]},
  'memory_free_gb': {'name': 'memory_free_gb',
   'x': [1789, 3900, 6184, 7946],
   'y': [11.314000129699707,
    11.312999725341797,
    11.314000129699707,
    11.314000129699707]},
  'memory_used_gb': {'name': 'memory_used_gb',
   'x': [1789, 3900, 6184, 79

# 🥾 Next steps

Now that you know the basics of ClearML, you can get started with running your own experiments!

## 📺 Check out our [Youtube Channel](https://www.youtube.com/@ClearML)!
It holds [detailed tutorial videos](https://www.youtube.com/watch?v=ZxgfHhPi8Gk&list=PLMdIlCuMqSTnoC45ME5_JnsJX0zWqDdlO), as well as [cool use cases and walkthroughs of real-life projects](https://www.youtube.com/watch?v=quSGXvuK1IM&list=PLMdIlCuMqSTmUfsAWXrK8zibwvxfduFPy)!

[![Getting Started](http://img.youtube.com/vi/s3k9ntmQmD4/0.jpg)](http://www.youtube.com/watch?v=s3k9ntmQmD4 "Getting Started")

[![Day in The Life Of](http://img.youtube.com/vi/quSGXvuK1IM/0.jpg)](http://www.youtube.com/watch?v=quSGXvuK1IM "Day in The Life Of")

[![Training a Sarcasm Detector](http://img.youtube.com/vi/0wmwnNnN8ow/0.jpg)](http://www.youtube.com/watch?v=0wmwnNnN8ow "Training a Sarcasm Detector")



## 💪 Check out our examples

We have very [specific feature examples](https://clear.ml/docs/latest/docs/clearml_sdk/task_sdk) as well as a repo full of [example scripts](https://github.com/allegroai/clearml/tree/master/examples)!

## 📚 Check out our [documentation](https://clear.ml/docs/latest/docs)

## 🔥 Check out our [blog](https://clear.ml/blog/)

